# Predicting the optimal placement of a hospital in a Toronto neighbourhood

## Table of Contents    
  * [Introduction: Business Problem](#intro)
  * [Data](#data)

## Introduction: Business Problem <a id="intro"></a>

<p> 
    For this project we will be trying to determine the best possible location to open an additional medical centre/ hospital in the city of Toronto. Many a time one might hear about news about how there may not be enough space at a hospital for all the patients needing help and when there comes a time like an epidemic or a pandemic (as is the current state of things in the world), this is especially the case. This report will likely be of interest to the board of directors and stakeholders of the hospital in question as well as the city of Toronto staff. 
    
</p>

<p>
    Our aim is to look for populated areas in neighbourhoods where there appears to be more young children and elderly present. Once those neighbourhoods have been found, we will then search the surrounding area for any other such health centres in the vicinity as we would like to construct the hospital in a distance far enough away from the others in an area where it would be most needed.
    
<p>

<p>
    Using this criteria and our purpose along with relevant data to provide support, we aim to share our findings and reasonings for our choices on where we would advise them to construct the health centre.
    
</p>

## Data <a id="data"></a>

<p> 
    As mentioned above in the business problem, the factors that will influence our decision are:
</p>
      
  * The number of hospitals/ health centres in the area
  * The number of people in the area
  * Ages of the constituents of the neighbourhood

<p> We will be using the following data sources for our analysis:</p>

  * **[Toronto neighbourhoods data](https://open.toronto.ca/dataset/neighbourhoods/)** - *Attribution: [Open Data Licence - Toronto](https://open.toronto.ca/open-data-license/); Contains information licensed under the Open Government Licence – Toronto* 

<p>This source will be used to obtain the initial neighbourhood profiles and their geographical coordinates</p>

  * **[Toronto neighbourhood profiles data](https://open.toronto.ca/dataset/neighbourhood-profiles/)** - *Attribution: [Open Data Licence - Toronto](https://open.toronto.ca/open-data-license/); Contains information licensed under the Open Government Licence – Toronto*
  
<p>This source will be used to determine the age of the constituents in their respective neighbourhoods</p>

  * **[Foursquare API](https://developer.foursquare.com/docs/)** 

<p>This API will be used to determine the location and number of hospitals in the Toronto area </p>